In [ ]:
# CryptoSignal Model Training Experiment
# Testing different train-test split ratios to optimize model performance

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import requests
import time
from datetime import datetime, timedelta

# Set up visual styling for our plots
plt.style.use('dark_background')
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

def fetch_coingecko_data(days=90):
    """Fetch price data from CoinGecko API"""
    try:
        url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
        params = {
            "vs_currency": "usd",
            "days": str(days),
            "interval": "daily"
        }
        
        print(f"Fetching {days} days of Bitcoin data from CoinGecko")
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            
            # Process price data
            prices = []
            for price_data in data.get('prices', []):
                timestamp = datetime.fromtimestamp(price_data[0]/1000)
                price = price_data[1]
                prices.append({
                    'timestamp': timestamp,
                    'price': price
                })
            
            # Process volume data
            volumes = []
            for volume_data in data.get('total_volumes', []):
                timestamp = datetime.fromtimestamp(volume_data[0]/1000)
                volume = volume_data[1]
                volumes.append({
                    'timestamp': timestamp,
                    'volume': volume
                })
            
            # Create DataFrame
            price_df = pd.DataFrame(prices)
            volume_df = pd.DataFrame(volumes)
            
            # Merge price and volume data
            df = pd.merge(price_df, volume_df, on='timestamp', how='outer')
            
            return df
        else:
            print(f"Failed to fetch CoinGecko data: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error in fetch_coingecko_data: {str(e)}")
        return None
    
# Function to fetch data similar to app.py but simplified for notebook
def fetch_coinbase_market_data(days=90):
    """Fetch market data from Coinbase API"""
    try:
        # Calculate time intervals
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        # Convert to ISO format
        start = start_date.isoformat()
        end = end_date.isoformat()
        
        # Coinbase API for BTC-USD historical data
        url = "https://api.exchange.coinbase.com/products/BTC-USD/candles"
        params = {
            "granularity": 86400,  # Daily candles (86400 seconds)
            "start": start,
            "end": end
        }
        
        # Add headers to avoid rate limiting
        headers = {
            "User-Agent": "TradingDashboardApp/1.0"
        }
        
        response = requests.get(url, params=params, headers=headers, timeout=30)
        
        if response.status_code == 200:
            candles = response.json()
            
            data = []
            for candle in candles:
                # Coinbase format: [timestamp, low, high, open, close, volume]
                timestamp = datetime.fromtimestamp(candle[0])
                low_price = float(candle[1])
                high_price = float(candle[2])
                open_price = float(candle[3])
                close_price = float(candle[4])
                volume = float(candle[5])
                
                data.append({
                    'timestamp': timestamp,
                    'price': close_price,
                    'open': open_price,
                    'high': high_price,
                    'low': low_price,
                    'volume': volume
                })
            
            # Sort by timestamp (Coinbase returns in reverse order)
            df = pd.DataFrame(data).sort_values('timestamp')
            return df
            
        else:
            print(f"Failed to fetch Coinbase data: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error fetching Coinbase market data: {str(e)}")
        return None

# Fetch and display raw data
print("Fetching Bitcoin price data...")
raw_data = fetch_coinbase_market_data(days=90)

if raw_data is None:
    print("Failed to fetch data from Coinbase. Trying alternate source...")
    
    # Your coingecko function would go here
    raw_data = fetch_coingecko_data(days=90)
    
    if raw_data is None:
        print("Failed to fetch data from all sources. Cannot proceed.")
        exit()

# 1. Display Raw Data Table
print("\n===== Raw Bitcoin Price Data =====")
print(f"Data shape: {raw_data.shape}")
print("\nFirst 5 rows:")
display(raw_data.head())

print("\nLast 5 rows:")
display(raw_data.tail())

# 2. Data Summary Statistics
print("\n===== Data Summary Statistics =====")
display(raw_data.describe())

# 3. Check for Missing Values
print("\n===== Missing Values Check =====")
missing_values = raw_data.isnull().sum()
print(f"Total missing values: {missing_values.sum()}")
display(missing_values)

# 4. Visualize Raw Price Data
plt.figure(figsize=(14, 7))
plt.plot(raw_data['timestamp'], raw_data['price'], label='Close Price', color='#4CAF50', linewidth=2)
plt.title('Bitcoin Price Over Time (Raw Data)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# 5. Visualize Trading Volume
plt.figure(figsize=(14, 7))
plt.bar(raw_data['timestamp'], raw_data['volume'], color='#2196F3', alpha=0.7)
plt.title('Bitcoin Trading Volume Over Time')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.grid(True, alpha=0.3)
plt.show()

# 6. Visualize Price Range (Candlestick-like)
plt.figure(figsize=(14, 7))
plt.fill_between(raw_data['timestamp'], raw_data['low'], raw_data['high'], 
                 alpha=0.3, color='#7B1FA2', label='Price Range (High-Low)')
plt.plot(raw_data['timestamp'], raw_data['price'], color='white', linewidth=1, label='Close Price')
plt.title('Bitcoin Price Range (High-Low)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# 7. Daily Returns Calculation and Visualization
raw_data['daily_returns'] = raw_data['price'].pct_change() * 100
raw_data['daily_returns'] = raw_data['daily_returns'].fillna(0)

plt.figure(figsize=(14, 7))
plt.bar(raw_data['timestamp'], raw_data['daily_returns'], 
        color=np.where(raw_data['daily_returns'] >= 0, '#4CAF50', '#F44336'))
plt.title('Bitcoin Daily Returns (%)')
plt.xlabel('Date')
plt.ylabel('Daily Return (%)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='white', linestyle='-', alpha=0.3)
plt.show()

# 8. Distribution of Daily Returns
plt.figure(figsize=(14, 7))
sns.histplot(raw_data['daily_returns'], bins=30, kde=True, color='#FFC107')
plt.title('Distribution of Daily Returns')
plt.xlabel('Daily Return (%)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.axvline(x=0, color='white', linestyle='--', alpha=0.5)
plt.show()

# 9. Correlation Analysis of Raw Features
raw_numeric_data = raw_data.select_dtypes(include=[np.number])
plt.figure(figsize=(10, 8))
correlation_matrix = raw_numeric_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Raw Features')
plt.tight_layout()
plt.show()

# Now proceed with Data Preprocessing
print("\n===== Data Preprocessing =====")

def process_data(df):
    """Process the data for use with the model"""
    if df is None:
        return None
    
    try:
        # Make sure 'timestamp' is a column, not an index
        if 'timestamp' not in df.columns and df.index.name == 'timestamp':
            df = df.reset_index()
        
        # Make sure we have a 'price' column
        price_cols = [col for col in df.columns if col.endswith('price') or col == 'price']
        if not price_cols and 'close' in df.columns:
            df = df.rename(columns={'close': 'price'})
        
        # Calculate additional features
        # 1. Returns
        print("Calculating returns...")
        df['returns'] = df['price'].pct_change().fillna(0)
        
        # 2. Volatility (7-day rolling standard deviation of returns)
        print("Calculating volatility...")
        df['volatility'] = df['returns'].rolling(window=7).std().fillna(0)
        
        # 3. Moving averages
        print("Calculating moving averages...")
        df['price_ma7'] = df['price'].rolling(window=7).mean().fillna(df['price'])
        df['price_ma30'] = df['price'].rolling(window=30).mean().fillna(df['price'])
        
        # 4. Momentum
        print("Calculating momentum...")
        df['momentum'] = (df['price'] / df['price_ma7'] - 1) * 100
        
        # 5. RSI calculation
        print("Calculating RSI...")
        delta = df['price'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        
        rs = avg_gain / avg_loss.replace(0, np.nan).fillna(1)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 6. Add market regime based on trend
        print("Determining market regimes...")
        if 'returns' in df.columns:
            # Simple regime based on returns trend
            df['market_regime'] = 0  # Neutral by default
            
            # Trending up: regime 1
            df.loc[df['returns'].rolling(10).mean() > 0.001, 'market_regime'] = 1
            
            # Trending down: regime 2
            df.loc[df['returns'].rolling(10).mean() < -0.001, 'market_regime'] = 2
        
        # 7. Add volume features if available
        if 'volume' in df.columns:
            print("Calculating volume features...")
            df['volume_change'] = df['volume'].pct_change().fillna(0)
            df['volume_ma7'] = df['volume'].rolling(window=7).mean().fillna(df['volume'])
            df['volume_momentum'] = (df['volume'] / df['volume_ma7'] - 1) * 100
        
        print(f"Processed data shape: {df.shape}")
        return df
    
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        return None

# Process the data
processed_df = process_data(raw_data)

# Display processed data
print("\n===== Processed Data =====")
print(f"Data shape after preprocessing: {processed_df.shape}")
print("\nNew features created:")
new_features = [col for col in processed_df.columns if col not in raw_data.columns]
print(", ".join(new_features))

print("\nFirst 5 rows of processed data:")
display(processed_df.head())

# 10. Visualize processed features
print("\n===== Visualizing Processed Features =====")

# Plot price with moving averages
plt.figure(figsize=(14, 7))
plt.plot(processed_df['timestamp'], processed_df['price'], label='Price', color='white', alpha=0.8)
plt.plot(processed_df['timestamp'], processed_df['price_ma7'], label='7-Day MA', color='#4CAF50', linewidth=2)
plt.plot(processed_df['timestamp'], processed_df['price_ma30'], label='30-Day MA', color='#2196F3', linewidth=2)
plt.title('Bitcoin Price with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Plot RSI
plt.figure(figsize=(14, 7))
plt.plot(processed_df['timestamp'], processed_df['rsi'], color='#FFC107', linewidth=2)
plt.axhline(y=70, color='#F44336', linestyle='--', alpha=0.5, label='Overbought (70)')
plt.axhline(y=30, color='#4CAF50', linestyle='--', alpha=0.5, label='Oversold (30)')
plt.title('Relative Strength Index (RSI)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim(0, 100)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# Plot market regimes
plt.figure(figsize=(14, 7))
plt.plot(processed_df['timestamp'], processed_df['price'], label='Price', color='white', alpha=0.6)

# Color regions based on market regime
for i in range(len(processed_df)-1):
    if processed_df['market_regime'].iloc[i] == 1:  # Bullish
        plt.axvspan(processed_df['timestamp'].iloc[i], processed_df['timestamp'].iloc[i+1], 
                   alpha=0.3, color='green')
    elif processed_df['market_regime'].iloc[i] == 2:  # Bearish
        plt.axvspan(processed_df['timestamp'].iloc[i], processed_df['timestamp'].iloc[i+1], 
                   alpha=0.3, color='red')

plt.title('Market Regimes')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

# 11. Correlation Analysis of Processed Features
processed_numeric_data = processed_df.select_dtypes(include=[np.number])
plt.figure(figsize=(14, 12))
correlation_matrix = processed_numeric_data.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Processed Features')
plt.tight_layout()
plt.show()

# 12. Feature Distribution
plt.figure(figsize=(14, 10))
for i, feature in enumerate(new_features[:min(9, len(new_features))]):
    plt.subplot(3, 3, i+1)
    sns.histplot(processed_df[feature], kde=True)
    plt.title(feature)
    plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 13. Time Series Decomposition (Trend, Seasonality, Residual)
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(processed_df['timestamp'], processed_df['price'], label='Original Price', color='white')
plt.title('Original Bitcoin Price')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(processed_df['timestamp'], processed_df['price_ma30'], label='Trend (30-day MA)', color='#4CAF50')
plt.title('Price Trend (30-day Moving Average)')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(3, 1, 3)
residual = processed_df['price'] - processed_df['price_ma30']
plt.plot(processed_df['timestamp'], residual, label='Residual', color='#F44336')
plt.title('Price Residual (Price - Trend)')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# 14. Summary of data preparation
print("\n===== Data Preparation Summary =====")
print(f"Total data points: {len(processed_df)}")
print(f"Time range: {processed_df['timestamp'].min()} to {processed_df['timestamp'].max()}")
print(f"Price range: ${processed_df['price'].min():.2f} to ${processed_df['price'].max():.2f}")
print(f"Average price: ${processed_df['price'].mean():.2f}")
print(f"Average daily return: {processed_df['returns'].mean()*100:.2f}%")
print(f"Market regime distribution:")
regime_counts = processed_df['market_regime'].value_counts()
for regime, count in regime_counts.items():
    regime_name = {0: 'Neutral', 1: 'Bullish', 2: 'Bearish'}.get(regime, 'Unknown')
    percentage = count / len(processed_df) * 100
    print(f"  - {regime_name}: {count} data points ({percentage:.1f}%)")

print("\nData is now ready for the train-test split experiment!")

# Function to fetch data from CoinGecko API as fallback
def fetch_coingecko_data(days=90):
    """Fetch price data from CoinGecko API"""
    try:
        # CoinGecko API for Bitcoin price history
        url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
        params = {
            "vs_currency": "usd",
            "days": str(days),
            "interval": "daily"
        }
        
        print(f"Fetching {days} days of Bitcoin data from CoinGecko")
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            
            # Process price data
            prices = []
            for price_data in data.get('prices', []):
                timestamp = datetime.fromtimestamp(price_data[0]/1000)
                price = price_data[1]
                prices.append({
                    'timestamp': timestamp,
                    'price': price
                })
            
            # Process volume data
            volumes = []
            for volume_data in data.get('total_volumes', []):
                timestamp = datetime.fromtimestamp(volume_data[0]/1000)
                volume = volume_data[1]
                volumes.append({
                    'timestamp': timestamp,
                    'volume': volume
                })
            
            # Create DataFrame
            price_df = pd.DataFrame(prices)
            volume_df = pd.DataFrame(volumes)
            
            # Merge price and volume data
            df = pd.merge(price_df, volume_df, on='timestamp', how='outer')
            
            return df
        else:
            print(f"Failed to fetch CoinGecko data: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error in fetch_coingecko_data: {str(e)}")
        return None

# Function to process data for model training similar to process_data() in app.py
def process_data(df):
    """Process the data for use with the model"""
    if df is None:
        return None
    
    try:
        # Make sure 'timestamp' is a column, not an index
        if 'timestamp' not in df.columns and df.index.name == 'timestamp':
            df = df.reset_index()
        
        # Make sure we have a 'price' column
        price_cols = [col for col in df.columns if col.endswith('price') or col == 'price']
        if not price_cols and 'close' in df.columns:
            df = df.rename(columns={'close': 'price'})
        
        # Calculate additional features
        if 'price' in df.columns:
            # Returns
            df['returns'] = df['price'].pct_change().fillna(0)
            
            # Volatility (7-day rolling standard deviation of returns)
            df['volatility'] = df['returns'].rolling(window=7).std().fillna(0)
            
            # Moving averages
            df['price_ma7'] = df['price'].rolling(window=7).mean().fillna(df['price'])
            df['price_ma30'] = df['price'].rolling(window=30).mean().fillna(df['price'])
            
            # Momentum
            df['momentum'] = (df['price'] / df['price_ma7'] - 1) * 100
            
            # RSI calculation
            delta = df['price'].diff()
            gain = (delta.where(delta > 0, 0)).fillna(0)
            loss = (-delta.where(delta < 0, 0)).fillna(0)
            
            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()
            
            rs = avg_gain / avg_loss.replace(0, np.nan).fillna(1)
            df['rsi'] = 100 - (100 / (1 + rs))
        
        # Add market regime based on trend
        if 'returns' in df.columns:
            # Simple regime based on returns trend
            df['market_regime'] = 0  # Neutral by default
            
            # Trending up: regime 1
            df.loc[df['returns'].rolling(10).mean() > 0.001, 'market_regime'] = 1
            
            # Trending down: regime 2
            df.loc[df['returns'].rolling(10).mean() < -0.001, 'market_regime'] = 2
        
        # Add volume features if available
        if 'volume' in df.columns:
            df['volume_change'] = df['volume'].pct_change().fillna(0)
            df['volume_ma7'] = df['volume'].rolling(window=7).mean().fillna(df['volume'])
            df['volume_momentum'] = (df['volume'] / df['volume_ma7'] - 1) * 100
        
        print(f"Processed data shape: {df.shape}")
        return df
    
    except Exception as e:
        print(f"Error processing data: {str(e)}")
        return None

# Experiment with different train-test split ratios
def build_model_with_ratio(df, window_size=7, target_col='returns', split_ratio=0.8):
    """Build a CNN-LSTM model with specific train-test split ratio"""
    if df is None or len(df) < window_size + 10:
        print(f"Insufficient data for model. Need at least {window_size + 10} rows")
        return None, None, None, None, None
    
    try:
        # Make sure we have the target column
        if target_col not in df.columns:
            if 'returns' in df.columns:
                target_col = 'returns'
            else:
                print(f"Target column '{target_col}' not found in data")
                return None, None, None, None, None
        
        print(f"Building model with target column: {target_col} and split ratio: {split_ratio}")
        
        # Drop non-numeric columns and timestamp
        numeric_df = df.select_dtypes(include=['number'])
        
        # Remove columns with all NaN or all same values
        cols_to_keep = [col for col in numeric_df.columns 
                       if not numeric_df[col].isna().all() 
                       and numeric_df[col].nunique() > 1]
        
        # Keep only relevant columns
        numeric_df = numeric_df[cols_to_keep]
        
        # Create target variable (next period's return)
        y = numeric_df[target_col].shift(-1).dropna()
        
        # Keep only rows that have a target value
        X = numeric_df.iloc[:len(y)]
        
        # Scale the features
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Prepare input sequences
        X_sequences = []
        y_values = []
        
        for i in range(len(X_scaled) - window_size):
            X_sequences.append(X_scaled[i:i+window_size])
            y_values.append(y.iloc[i+window_size])
        
        # Convert to numpy arrays
        X_sequences = np.array(X_sequences)
        y_values = np.array(y_values)
        
        # Split into training and testing sets
        split_idx = int(len(X_sequences) * split_ratio)
        X_train, X_test = X_sequences[:split_idx], X_sequences[split_idx:]
        y_train, y_test = y_values[:split_idx], y_values[split_idx:]
        
        # Build CNN-LSTM model
        model = tf.keras.Sequential([
            # CNN layers for feature extraction
            Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(window_size, X.shape[1])),
            MaxPooling1D(pool_size=2),
            
            # LSTM layers for sequence learning
            LSTM(50, return_sequences=True),
            Dropout(0.2),
            LSTM(50),
            Dropout(0.2),
            
            # Dense output layers
            Dense(25, activation='relu'),
            Dense(1)  # Prediction output
        ])
        
        # Compile the model
        model.compile(optimizer='adam', loss='mse')
        
        # Train the model with early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
        
        print(f"Training the CNN-LSTM model with {len(X_train)} training samples and {len(X_test)} test samples...")
        history = model.fit(
            X_train, y_train,
            epochs=50,  # Maximum epochs
            batch_size=32,
            validation_data=(X_test, y_test),
            callbacks=[early_stopping],
            verbose=1
        )
        
        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Model evaluation with {split_ratio:.1f} split ratio:")
        print(f"MSE: {mse:.6f}")
        print(f"MAE: {mae:.6f}")
        print(f"R²: {r2:.6f}")
        
        return model, scaler, X.columns, history, {
            'X_train': X_train, 'y_train': y_train,
            'X_test': X_test, 'y_test': y_test,
            'y_pred': y_pred, 'mse': mse, 'mae': mae, 'r2': r2
        }
    
    except Exception as e:
        print(f"Error building model: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None, None, None

# Function to generate trading signals based on the model
def generate_signals(model, scaler, df, features, window_size=7, threshold=0.001):
    """Generate trading signals based on model predictions"""
    if model is None or scaler is None or df is None:
        print("Missing required inputs for signal generation")
        return None
    
    try:
        # Make a copy of the dataframe
        signal_df = df.copy()
        
        # Select features used in training
        X = signal_df[features].copy()
        
        # Scale the features
        X_scaled = scaler.transform(X)
        
        # Generate predictions for each possible window
        predictions = []
        timestamps = []
        
        for i in range(len(X_scaled) - window_size + 1):
            window_data = X_scaled[i:i+window_size].reshape(1, window_size, len(features))
            pred = model.predict(window_data, verbose=0)[0][0]
            predictions.append(pred)
            timestamps.append(signal_df['timestamp'].iloc[i+window_size-1])
        
        # Create a prediction dataframe
        pred_df = pd.DataFrame({
            'timestamp': timestamps,
            'predicted_return': predictions
        })
        
        # Generate signals based on predicted returns
        pred_df['signal'] = 'HOLD'
        pred_df.loc[pred_df['predicted_return'] > threshold, 'signal'] = 'BUY'
        pred_df.loc[pred_df['predicted_return'] < -threshold, 'signal'] = 'SELL'
        
        # Calculate signal confidence
        pred_df['confidence'] = abs(pred_df['predicted_return']) * 100
        
        # Merge signals back with original data
        result_df = pd.merge(signal_df, pred_df, on='timestamp', how='left')
        
        # Forward fill signals for any missing values
        result_df['signal'] = result_df['signal'].fillna('HOLD')
        result_df['confidence'] = result_df['confidence'].fillna(0)
        
        return result_df
    
    except Exception as e:
        print(f"Error generating signals: {str(e)}")
        return None

# Function to evaluate trading strategy performance
def evaluate_performance(signals_df, price_col='price'):
    """Evaluate trading strategy performance metrics with various criteria"""
    if signals_df is None:
        print("No signals dataframe provided for evaluation")
        return None
    
    try:
        # Verify signal column exists
        if 'signal' not in signals_df.columns:
            print("Signal column not found in dataframe")
            return None
        
        # Find price column if not specified
        if price_col not in signals_df.columns:
            price_cols = [col for col in signals_df.columns if col.endswith('price') or col == 'price']
            if not price_cols:
                print("No price column found for performance evaluation")
                return None
            price_col = price_cols[0]
        
        # Find or create returns column
        returns_col = 'returns'
        if returns_col not in signals_df.columns:
            signals_df[returns_col] = signals_df[price_col].pct_change().fillna(0)
        
        # Calculate strategy positions
        signals_df['position'] = 0
        signals_df.loc[signals_df['signal'] == 'BUY', 'position'] = 1
        signals_df.loc[signals_df['signal'] == 'SELL', 'position'] = -1
        
        # Shift positions (implement on next period)
        signals_df['position'] = signals_df['position'].shift(1).fillna(0)
        
        # Calculate strategy returns
        signals_df['strategy_return'] = signals_df['position'] * signals_df[returns_col]
        
        # Calculate cumulative returns
        signals_df['cumulative_return'] = (1 + signals_df['strategy_return']).cumprod()
        
        # Calculate buy and hold returns
        signals_df['buy_hold_return'] = (1 + signals_df[returns_col]).cumprod()
        
        # Calculate performance metrics
        # Sharpe Ratio (assuming daily data)
        annualization_factor = np.sqrt(252)
        
        mean_return = signals_df['strategy_return'].mean()
        std_return = max(signals_df['strategy_return'].std(), 0.0001)  # Avoid division by zero
        sharpe_ratio = mean_return / std_return * annualization_factor
        
        # Maximum Drawdown
        cum_returns = signals_df['cumulative_return']
        if len(cum_returns) > 0:
            rolling_max = cum_returns.cummax()
            drawdown = (cum_returns / rolling_max - 1)
            max_drawdown = abs(drawdown.min())
        else:
            max_drawdown = 0
        
        # Trade Frequency
        position_changes = signals_df['position'].diff() != 0
        trade_frequency = position_changes.sum() / len(signals_df)
        
        # Signal counts
        buy_signals = (signals_df['signal'] == 'BUY').sum()
        sell_signals = (signals_df['signal'] == 'SELL').sum()
        hold_signals = (signals_df['signal'] == 'HOLD').sum()
        
        # Final performance
        final_return = signals_df['cumulative_return'].iloc[-1] - 1
        buy_hold_return = signals_df['buy_hold_return'].iloc[-1] - 1
        
        # Store performance metrics
        performance = {
            'sharpe_ratio': float(sharpe_ratio),
            'max_drawdown': float(max_drawdown),
            'trade_frequency': float(trade_frequency),
            'buy_signals': int(buy_signals),
            'sell_signals': int(sell_signals),
            'hold_signals': int(hold_signals),
            'final_return': float(final_return),
            'buy_hold_return': float(buy_hold_return),
            'outperformance': float(final_return - buy_hold_return),
            'meets_sharpe_ratio': bool(sharpe_ratio >= 1.8),
            'meets_drawdown': bool(max_drawdown <= 0.4),
            'meets_frequency': bool(trade_frequency >= 0.03),
            'meets_criteria': bool(sharpe_ratio >= 1.8 and max_drawdown <= 0.4 and trade_frequency >= 0.03)
        }
        
        return performance, signals_df
    
    except Exception as e:
        print(f"Error evaluating performance: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

# Function to visualize training history
def plot_training_history(history, title='Model Training History'):
    """Plot training and validation loss"""
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    return plt

# Function to visualize predictions vs actual values
def plot_predictions(y_test, y_pred, title='Model Predictions vs Actual Values'):
    """Plot test predictions against actual values"""
    plt.figure(figsize=(12, 6))
    plt.plot(y_test, label='Actual Returns', alpha=0.7)
    plt.plot(y_pred, label='Predicted Returns', alpha=0.7)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Returns')
    plt.legend()
    plt.grid(True, alpha=0.3)
    return plt

# Function to visualize trading signals and cumulative returns
def plot_trading_strategy(signals_df, title='Trading Strategy Performance'):
    """Plot price, signals, and cumulative returns"""
    # Create a figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    
    # Plot price on first subplot
    ax1.plot(signals_df['timestamp'], signals_df['price'], label='Price', color='white', alpha=0.8)
    ax1.set_title('Bitcoin Price with Trading Signals')
    ax1.set_ylabel('Price (USD)')
    ax1.grid(True, alpha=0.3)
    
    # Plot buy signals
    buy_signals = signals_df[signals_df['signal'] == 'BUY']
    ax1.scatter(buy_signals['timestamp'], buy_signals['price'], 
               marker='^', color='green', s=100, label='Buy Signal')
    
    # Plot sell signals
    sell_signals = signals_df[signals_df['signal'] == 'SELL']
    ax1.scatter(sell_signals['timestamp'], sell_signals['price'], 
               marker='v', color='red', s=100, label='Sell Signal')
    
    ax1.legend()
    
    # Plot cumulative returns on second subplot
    ax2.plot(signals_df['timestamp'], signals_df['cumulative_return'], 
            label='Strategy Returns', color='#4CAF50', linewidth=2)
    ax2.plot(signals_df['timestamp'], signals_df['buy_hold_return'], 
            label='Buy & Hold Returns', color='#2196F3', linewidth=2, alpha=0.7)
    ax2.set_title('Cumulative Returns')
    ax2.set_ylabel('Returns')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    return fig, (ax1, ax2)

# Function to visualize performance metrics across different split ratios
def plot_performance_comparison(performance_metrics, split_ratios):
    """Plot performance metrics for different train-test split ratios"""
    metrics = ['sharpe_ratio', 'max_drawdown', 'trade_frequency', 'final_return']
    titles = ['Sharpe Ratio', 'Maximum Drawdown', 'Trade Frequency', 'Final Return']
    colors = ['#4CAF50', '#F44336', '#2196F3', '#FFC107']
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for i, (metric, title, color) in enumerate(zip(metrics, titles, colors)):
        values = [metrics[metric] for metrics in performance_metrics]
        axes[i].bar(split_ratios, values, color=color, alpha=0.7)
        axes[i].set_title(title)
        axes[i].set_xlabel('Train-Test Split Ratio')
        axes[i].set_ylabel(title)
        axes[i].grid(True, alpha=0.3)
        
        # Add value labels on top of bars
        for j, value in enumerate(values):
            axes[i].text(j, value, f'{value:.4f}', ha='center', va='bottom')
    
    plt.tight_layout()
    return fig, axes

# Main experiment function to run the train-test ratio comparison
def run_train_test_ratio_experiment():
    """Run experiment with different train-test split ratios"""
    print("Starting train-test split ratio experiment...")
    
    # Fetch and process data
    print("Fetching data...")
    df = fetch_coinbase_market_data(days=90)
    
    if df is None:
        print("Trying alternative data source...")
        df = fetch_coingecko_data(days=90)
    
    if df is None:
        print("Failed to fetch data from any source. Cannot proceed with experiment.")
        return None
    
    # Process data
    print("Processing data...")
    processed_df = process_data(df)
    
    if processed_df is None:
        print("Failed to process data. Cannot proceed with experiment.")
        return None
    
    # Define split ratios to test
    split_ratios = [0.7, 0.8, 0.9]
    
    # Store results
    results = []
    performance_metrics = []
    
    # Run experiment for each split ratio
    for ratio in split_ratios:
        print(f"\n{'='*50}")
        print(f"Testing train-test split ratio: {ratio}")
        print(f"{'='*50}")
        
        # Build and train model
        model, scaler, features, history, evaluation = build_model_with_ratio(
            processed_df, window_size=7, target_col='returns', split_ratio=ratio
        )
        
        if model is None:
            print(f"Failed to build model with ratio {ratio}. Skipping...")
            continue
        
        # Generate signals
        signals_df = generate_signals(model, scaler, processed_df, features)
        
        if signals_df is None:
            print(f"Failed to generate signals with ratio {ratio}. Skipping...")
            continue
        
        # Evaluate performance
        performance, signals_with_returns = evaluate_performance(signals_df)
        
        if performance is None:
            print(f"Failed to evaluate performance with ratio {ratio}. Skipping...")
            continue
        
        # Store results
        results.append({
            'ratio': ratio,
            'model': model,
            'scaler': scaler,
            'features': features,
            'history': history,
            'evaluation': evaluation,
            'signals_df': signals_with_returns,
            'performance': performance
        })
        
        performance_metrics.append(performance)
        
        # Print performance summary
        print(f"\nPerformance Summary for {ratio} split ratio:")
        print(f"Sharpe Ratio: {performance['sharpe_ratio']:.4f}")
        print(f"Maximum Drawdown: {performance['max_drawdown']:.4f}")
        print(f"Trade Frequency: {performance['trade_frequency']:.4f}")
        print(f"Final Return: {performance['final_return']:.4f}")
        print(f"Buy & Hold Return: {performance['buy_hold_return']:.4f}")
        print(f"Outperformance: {performance['outperformance']:.4f}")
        print(f"Signal Counts: Buy={performance['buy_signals']}, Sell={performance['sell_signals']}, Hold={performance['hold_signals']}")
        print(f"Meets Criteria: {performance['meets_criteria']}")
    
    if not results:
        print("No successful experiments. Cannot compare results.")
        return None
    
    # Compare results and select best model
    best_result = max(results, key=lambda x: x['performance']['sharpe_ratio'])
    best_ratio = best_result['ratio']
    
    print(f"\n{'='*50}")
    print(f"Best Split Ratio: {best_ratio}")
    print(f"{'='*50}")
    print(f"Sharpe Ratio: {best_result['performance']['sharpe_ratio']:.4f}")
    print(f"Maximum Drawdown: {best_result['performance']['max_drawdown']:.4f}")
    print(f"Trade Frequency: {best_result['performance']['trade_frequency']:.4f}")
    print(f"Final Return: {best_result['performance']['final_return']:.4f}")
    
    return {
        'results': results,
        'performance_metrics': performance_metrics,
        'split_ratios': split_ratios,
        'best_result': best_result,
        'best_ratio': best_ratio,
        'processed_df': processed_df
    }

# Run the experiment
experiment_results = run_train_test_ratio_experiment()

def visualize_ratio_comparison(experiment_results):
    """
    Create comprehensive visualizations comparing the performance metrics
    across different train-test split ratios
    """
    if not experiment_results:
        print("No experiment results to visualize")
        return
    
    split_ratios = experiment_results['split_ratios']
    performance_metrics = experiment_results['performance_metrics']
    best_ratio = experiment_results['best_ratio']
    
    # Create a DataFrame for easier manipulation
    metrics_df = pd.DataFrame(performance_metrics)
    metrics_df['split_ratio'] = split_ratios
    
    # Set split_ratio as index for better display
    metrics_df = metrics_df.set_index('split_ratio')
    
    # 1. Tabular Comparison of All Metrics
    print("\n===== Train-Test Split Ratio Comparison (Tabular) =====")
    comparison_cols = [
        'sharpe_ratio', 'max_drawdown', 'trade_frequency', 
        'final_return', 'buy_hold_return', 'outperformance',
        'buy_signals', 'sell_signals', 'hold_signals'
    ]
    
    # Highlight the best ratio with a custom formatter
    def highlight_best(s):
        is_best = pd.Series(data=False, index=s.index)
        if s.name == 'sharpe_ratio' or s.name == 'final_return' or s.name == 'outperformance':
            is_best = s == s.max()
        elif s.name == 'max_drawdown':
            is_best = s == s.min()
        return ['background-color: #4CAF50; color: white' if v else '' for v in is_best]
    
    # Display styled DataFrame with conditional formatting
    styled_df = metrics_df[comparison_cols].style.apply(highlight_best)
    display(styled_df)
    
    # 2. Bar Chart Comparison for Key Metrics
    key_metrics = ['sharpe_ratio', 'max_drawdown', 'trade_frequency', 'final_return']
    titles = ['Sharpe Ratio', 'Maximum Drawdown', 'Trade Frequency', 'Final Return']
    colors = ['#4CAF50', '#F44336', '#2196F3', '#FFC107']
    
    plt.figure(figsize=(16, 12))
    for i, (metric, title, color) in enumerate(zip(key_metrics, titles, colors)):
        plt.subplot(2, 2, i+1)
        
        # Get values for each ratio
        values = [metrics[metric] for metrics in performance_metrics]
        
        # Create bar chart
        bars = plt.bar(split_ratios, values, color=color, alpha=0.7)
        
        # Highlight the best ratio
        best_idx = values.index(max(values)) if metric != 'max_drawdown' else values.index(min(values))
        bars[best_idx].set_color('gold')
        bars[best_idx].set_alpha(1.0)
        
        # Add value labels on top of bars
        for j, value in enumerate(values):
            plt.text(j, value, f'{value:.4f}', ha='center', va='bottom')
        
        # Add target lines if applicable
        if metric == 'sharpe_ratio':
            plt.axhline(y=1.8, color='white', linestyle='--', alpha=0.5, label='Target: ≥ 1.8')
        elif metric == 'max_drawdown':
            plt.axhline(y=0.4, color='white', linestyle='--', alpha=0.5, label='Target: ≤ 0.4')
        elif metric == 'trade_frequency':
            plt.axhline(y=0.03, color='white', linestyle='--', alpha=0.5, label='Target: ≥ 0.03')
        
        plt.title(title)
        plt.xlabel('Train-Test Split Ratio')
        plt.ylabel(title)
        plt.xticks(range(len(split_ratios)), split_ratios)
        plt.grid(True, alpha=0.3)
        if metric in ['sharpe_ratio', 'max_drawdown', 'trade_frequency']:
            plt.legend()
    
    plt.tight_layout()
    plt.suptitle('Comparison of Key Metrics Across Different Train-Test Split Ratios', 
                fontsize=16, y=1.02)
    plt.show()
    
    # 3. Radar Chart for Multi-dimensional Comparison
    plt.figure(figsize=(10, 10))
    
    # Prepare data for radar chart
    categories = ['Sharpe Ratio', 'Final Return', 'Trade Frequency', 
                 'Signal Accuracy', 'Computational\nEfficiency']
    
    # Normalize metrics to 0-1 scale for radar chart
    normalized_metrics = []
    
    for ratio_idx, ratio in enumerate(split_ratios):
        # Extract metrics for this ratio
        ratio_metrics = performance_metrics[ratio_idx]
        
        # Normalize each metric (higher is better for all)
        sharpe_ratio = ratio_metrics['sharpe_ratio'] / max([m['sharpe_ratio'] for m in performance_metrics])
        final_return = ratio_metrics['final_return'] / max([m['final_return'] for m in performance_metrics])
        trade_freq = ratio_metrics['trade_frequency'] / max([m['trade_frequency'] for m in performance_metrics])
        
        # Invert max_drawdown since lower is better
        max_dd_values = [m['max_drawdown'] for m in performance_metrics]
        signal_accuracy = 1 - (ratio_metrics['max_drawdown'] / max(max_dd_values))
        
        # Computational efficiency (more training data = lower efficiency)
        comp_efficiency = 1 - ((ratio - min(split_ratios)) / (max(split_ratios) - min(split_ratios)))
        
        normalized_metrics.append([sharpe_ratio, final_return, trade_freq, signal_accuracy, comp_efficiency])
    
    # Set up radar chart
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]  # Close the loop
    
    ax = plt.subplot(111, polar=True)
    
    # Add lines for each ratio
    for idx, ratio in enumerate(split_ratios):
        values = normalized_metrics[idx]
        values += values[:1]  # Close the loop
        
        color = 'gold' if ratio == best_ratio else f'C{idx}'
        linewidth = 3 if ratio == best_ratio else 2
        alpha = 1.0 if ratio == best_ratio else 0.7
        
        ax.plot(angles, values, 'o-', linewidth=linewidth, 
                color=color, label=f'{ratio*100:.0f}//{(1-ratio)*100:.0f} Split', alpha=alpha)
        ax.fill(angles, values, color=color, alpha=0.1)
    
    # Set up radar chart properties
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'])
    ax.set_ylim(0, 1)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    plt.title('Multi-dimensional Comparison of Train-Test Split Ratios', size=15)
    plt.tight_layout()
    plt.show()
    
    # 4. Line Chart Comparison of Cumulative Returns
    plt.figure(figsize=(14, 7))
    
    # Get signals dataframes for each ratio
    best_idx = split_ratios.index(best_ratio)
    
    # Loop through ratios and plot cumulative returns
    for idx, ratio in enumerate(split_ratios):
        signals_df = experiment_results['results'][idx]['signals_df']
        
        # Plot strategy returns
        linewidth = 3 if ratio == best_ratio else 2
        alpha = 1.0 if ratio == best_ratio else 0.7
        
        plt.plot(signals_df['timestamp'], signals_df['cumulative_return'], 
                linewidth=linewidth, alpha=alpha,
                label=f'{ratio*100:.0f}//{(1-ratio)*100:.0f} Split')
    
    # Add buy and hold returns from best model for comparison
    best_signals_df = experiment_results['results'][best_idx]['signals_df']
    plt.plot(best_signals_df['timestamp'], best_signals_df['buy_hold_return'], 
            linewidth=2, linestyle='--', color='white', alpha=0.7,
            label='Buy & Hold')
    
    plt.title('Cumulative Returns Comparison Across Different Train-Test Split Ratios')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    # 5. Signal Distribution Comparison
    plt.figure(figsize=(16, 5))
    
    for idx, ratio in enumerate(split_ratios):
        signals_df = experiment_results['results'][idx]['signals_df']
        signal_counts = signals_df['signal'].value_counts()
        
        # Create pie chart
        plt.subplot(1, len(split_ratios), idx+1)
        
        colors = ['#4CAF50', '#F44336', '#FFC107']  # Green, Red, Yellow
        explode = [0.1 if signal in ['BUY', 'SELL'] else 0 for signal in signal_counts.index]
        
        plt.pie(signal_counts, labels=signal_counts.index, autopct='%1.1f%%', 
                colors=colors[:len(signal_counts)], startangle=90, explode=explode)
        plt.title(f'Signal Distribution\n{ratio*100:.0f}//{(1-ratio)*100:.0f} Split')
        plt.axis('equal')
    
    plt.tight_layout()
    plt.show()
    
    # 6. Summary Table with Recommendations
    print("\n===== Train-Test Split Ratio Experiment Summary =====")
    print(f"Best Ratio: {best_ratio*100:.0f}/{(1-best_ratio)*100:.0f} Split")
    
    # Display metrics for the best ratio
    best_perf = experiment_results['best_result']['performance']
    
    print(f"\nPerformance Metrics for {best_ratio*100:.0f}/{(1-best_ratio)*100:.0f} Split:")
    print(f"- Sharpe Ratio: {best_perf['sharpe_ratio']:.4f}")
    print(f"- Maximum Drawdown: {best_perf['max_drawdown']:.4f}")
    print(f"- Trade Frequency: {best_perf['trade_frequency']:.4f}")
    print(f"- Final Return: {best_perf['final_return']*100:.2f}%")
    print(f"- Buy & Hold Return: {best_perf['buy_hold_return']*100:.2f}%")
    print(f"- Outperformance: {best_perf['outperformance']*100:.2f}%")
    
    # Recommendations
    print("\nRecommendations:")
    print(f"1. Use the {best_ratio*100:.0f}/{(1-best_ratio)*100:.0f} split ratio for optimal model performance")
    
    # Specific recommendations based on the results
    if best_ratio == 0.7:
        print("2. This ratio provides more test data, which is beneficial for robust evaluation")
        print("3. Consider ensemble methods to improve performance with this lower training data ratio")
    elif best_ratio == 0.8:
        print("2. This ratio provides a balanced approach between training and testing data")
        print("3. For optimal results, use a periodic model retraining schedule to adapt to new market conditions")
    elif best_ratio == 0.9:
        print("2. This ratio maximizes training data, capturing more market patterns")
        print("3. Consider cross-validation techniques to ensure model generalizability with limited test data")
    
    # General recommendation
    print("4. Implement a sliding window approach in production to continuously retrain the model")
    
    return best_ratio

# Call the visualization function after running the experiment
if 'experiment_results' in locals() and experiment_results:
    best_ratio = visualize_ratio_comparison(experiment_results)
    print(f"\nThe optimal train-test split ratio is: {best_ratio}")
 
    